In [1]:
# Run this cell to install the required packages in the Jupyter notebook environment.
# pip install pandas google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client python-dotenv email-validator

In [2]:
import os
import pandas as pd
import base64
import pickle
from google.oauth2.service_account import Credentials as service_credentials
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.errors import HttpError
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from dotenv import load_dotenv
import csv

# Retrieve variables from .env
SERVICE_ACCOUNT_FILE = os.getenv('SERVICE_ACCOUNT_FILE')
SPREADSHEET_ID = os.getenv('SPREADSHEET_ID')
SAMPLE_RANGE_NAME = 'Form Responses 1'  # Update the range if necessary
# The scopes required by the Sheets and Drive APIs
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

In [3]:
# Replace this with the path to your actual CSV file
csv_file_path = 'headcount.csv'

# Initialize a dictionary to store sub-teams and their headcounts
team_headcounts = {}

# Open the CSV file and read it
with open(csv_file_path, newline='') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    next(csvreader, None)  # Skip the header row
    
    for row in csvreader:
        # Assuming the sub-team is in the first column and the headcount is in the second
        sub_team = row[0]
        headcount = int(row[1])  # Convert headcount to integer

        # Store the sub-team and headcount in the dictionary
        team_headcounts[sub_team] = headcount

## GOOGLE SHEET

In [4]:
def get_google_sheet(service_account_file, spreadsheet_id, range_name, fill_value='N/A'):
    """Retrieve sheet data using OAuth credentials and Google Python API, 
    filling missing values with a specified fill_value."""
    creds = service_credentials.from_service_account_file(service_account_file)
    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=spreadsheet_id, range=range_name).execute()
    values = result.get('values', [])

    if not values:
        print("No data found.")
        return []

    # Ensure all rows have the same number of columns
    num_columns = len(values[0])  # Number of columns is based on the first row (header row)
    for row in values:
        # while len(row) < num_columns:
        #     row.append(fill_value)  # Append fill_value to rows with fewer elements than headers
        row.extend([fill_value] * (num_columns - len(row)))

    return values

In [5]:
values = get_google_sheet(SERVICE_ACCOUNT_FILE, SPREADSHEET_ID, SAMPLE_RANGE_NAME)
whole_df = pd.DataFrame(values[1:], columns=values[0])  # Using the first row as column names

def replace_empty_with_na(cell):
    if isinstance(cell, str) and len(cell) == 0:  # Check if the cell is a string and empty
        return "N/A"
    return cell

whole_df = whole_df.applymap(replace_empty_with_na)

needsTeam_df = whole_df[whole_df['Team Assigned'] == "N/A"]

C:\Users\manyu\AppData\Local\Temp\ipykernel_15064\4148001816.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  whole_df = whole_df.applymap(replace_empty_with_na)


In [6]:
whole_df

,Timestamp,Email Address,First Name,Last Name,Team preference 1,Team preference 2,Team preference 3,Team Assigned,Accept/Decline,Team,Sub-Team,Title,of_status,upa_id,upa_status,pw_id,pw_status,onboarding_complete
0,2/17/2024 14:58:05,ahmedirtija8@gmail.com,aH,SDV,PCM - 1,EiM - 2,Cavs - 1,Comms - 2,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1,2/17/2024 14:59:32,airtija@ucdavis.edu,d,d,SDI - 1,EiM - 3,Cavs - 2,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
2,2/17/2024 14:59:35,ahmedirtija2001@gmail.com,jnsf,hgvhg,SDI - 1,PCM - 1,Cavs - 2,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
3,2/17/2024 14:59:36,example@gmail.com,jnsfd,dccdc,EiM - 3,PCM - 1,Cavs - 2,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
4,2/17/2024 14:59:38,example2@gmail.com,jnsfd,dccdc,Cavs - 1,PCM - 1,Cavs - 2,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
5,3/7/2024 16:19:53,ahmedirtija8@gmail.com,Test,FG,PM - 2,Comms - 2,EiM - 1,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A


In [7]:
needsTeam_df

,Timestamp,Email Address,First Name,Last Name,Team preference 1,Team preference 2,Team preference 3,Team Assigned,Accept/Decline,Team,Sub-Team,Title,of_status,upa_id,upa_status,pw_id,pw_status,onboarding_complete
1,2/17/2024 14:59:32,airtija@ucdavis.edu,d,d,SDI - 1,EiM - 3,Cavs - 2,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
2,2/17/2024 14:59:35,ahmedirtija2001@gmail.com,jnsf,hgvhg,SDI - 1,PCM - 1,Cavs - 2,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
3,2/17/2024 14:59:36,example@gmail.com,jnsfd,dccdc,EiM - 3,PCM - 1,Cavs - 2,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
4,2/17/2024 14:59:38,example2@gmail.com,jnsfd,dccdc,Cavs - 1,PCM - 1,Cavs - 2,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
5,3/7/2024 16:19:53,ahmedirtija8@gmail.com,Test,FG,PM - 2,Comms - 2,EiM - 1,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A


In [8]:
def assignMembers(needsTeam_df, team_headcounts, csv_file_path):
    assigned_df = pd.DataFrame([], columns=values[0])
    index = 0

    for i in range(1, 4):
        for idx, row in needsTeam_df.iterrows():
            team = row[f'Team preference {i}']
            if team_headcounts[team] > 0 and row["Team Assigned"] == "N/A":
                assigned_df = pd.concat([assigned_df, pd.DataFrame([row])], ignore_index=True)
                assigned_df.at[index, 'Team Assigned'] = team
                needsTeam_df = needsTeam_df.drop(idx)
                team_headcounts[team] -= 1
                index += 1
    
    # Update the CSV file with new headcounts
    with open(csv_file_path, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Sub-team', 'headcount'])  # Assuming these are your column headers
        for team, count in team_headcounts.items():
            csvwriter.writerow([team, count])

    return assigned_df, needsTeam_df

In [9]:
assigned_members_df, unnassigned_members_df = assignMembers(needsTeam_df, team_headcounts, csv_file_path)

In [10]:
assigned_members_df

,Timestamp,Email Address,First Name,Last Name,Team preference 1,Team preference 2,Team preference 3,Team Assigned,Accept/Decline,Team,Sub-Team,Title,of_status,upa_id,upa_status,pw_id,pw_status,onboarding_complete
0,3/7/2024 16:19:53,ahmedirtija8@gmail.com,Test,FG,PM - 2,Comms - 2,EiM - 1,Comms - 2,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A


In [11]:
unnassigned_members_df

,Timestamp,Email Address,First Name,Last Name,Team preference 1,Team preference 2,Team preference 3,Team Assigned,Accept/Decline,Team,Sub-Team,Title,of_status,upa_id,upa_status,pw_id,pw_status,onboarding_complete
1,2/17/2024 14:59:32,airtija@ucdavis.edu,d,d,SDI - 1,EiM - 3,Cavs - 2,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
2,2/17/2024 14:59:35,ahmedirtija2001@gmail.com,jnsf,hgvhg,SDI - 1,PCM - 1,Cavs - 2,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
3,2/17/2024 14:59:36,example@gmail.com,jnsfd,dccdc,EiM - 3,PCM - 1,Cavs - 2,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
4,2/17/2024 14:59:38,example2@gmail.com,jnsfd,dccdc,Cavs - 1,PCM - 1,Cavs - 2,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A


In [12]:
def write_team_assignments_to_sheet(assigned_members_df, whole_df, service_account_file, spreadsheet_id):
    """Write team assignments to the Google Sheet based on assigned members dataframe."""
    creds = service_credentials.from_service_account_file(service_account_file, scopes=['https://www.googleapis.com/auth/spreadsheets'])
    service = build('sheets', 'v4', credentials=creds)

    # No need to fetch current values from the sheet as we are using the index from df
    update_data = []
    for _, assigned_row in assigned_members_df.iterrows():
        email_address = assigned_row['Email Address']
        team_assigned = assigned_row['Team Assigned']
        
        # Find the index of the row in df that matches the assigned member's email address
        index_in_df = whole_df.index[whole_df['Email Address'] == email_address].tolist()[0] + 2  # +2 to adjust for Google Sheets indexing and header
        
        update_data.append({
            'range': f'H{index_in_df}',  # Writing to column H
            'values': [[team_assigned]]
        })

    # Batch update the values
    body = {
        'valueInputOption': 'USER_ENTERED',
        'data': update_data
    }
    service.spreadsheets().values().batchUpdate(spreadsheetId=spreadsheet_id, body=body).execute()

In [13]:
write_team_assignments_to_sheet(assigned_members_df, whole_df, SERVICE_ACCOUNT_FILE, SPREADSHEET_ID)

## GMAIL

In [14]:
# Create email message
def create_message(sender, to, subject, message_text):
    message = MIMEMultipart()
    message['from'] = sender
    message['to'] = to
    message['subject'] = subject
    message.attach(MIMEText(message_text, 'plain'))
    raw_msg = base64.urlsafe_b64encode(message.as_bytes()).decode()
    return {'raw': raw_msg}

# Email sending function
def send_email(service, user_id, message):
    try:
        message = service.users().messages().send(userId=user_id, body=message).execute()
        print(f"Email sent to {message['id']}")
    except HttpError as error:
        print(f'An error occurred: {error}')


# Initialize Gmail API
def get_gmail_service(client_secret_file, scopes):
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(client_secret_file, scopes)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return build('gmail', 'v1', credentials=creds)

In [15]:
# Map of team names to team leader emails
team_leader_emails = {
    "Comms - 1": "fakeemail1@example.com",
    "Comms - 2": "fakeemail2@example.com",
    "Comms -3": "fakeemail3@example.com",
    "SDI - 1": "fakeemail4@example.com",
    "SDI -2": "fakeemail5@example.com",
    "SDI -3": "fakeemail6@example.com",
    "PCM - 1": "fakeemail7@example.com",
    "PCM -2": "fakeemail8@example.com",
    "PM - 1": "fakeemail9@example.com",
    "PM - 2": "fakeemail10@example.com",
    "EiM - 1": "fakeemail11@example.com",
    "EiM - 2": "fakeemail12@example.com",
    "EiM - 3": "fakeemail13@example.com",
    "Cavs - 1": "fakeemail14@example.com",
    "Cavs - 2": "fakeemail15@example.com",
    "Cavs - 3": "fakeemail16@example.com"
}

In [16]:
client_secret_file = os.getenv('CLIENT_SECRET_FILE')
sender_email = os.getenv('SENDER_EMAIL')

In [17]:
# Function to notify team leaders
def notify_team_leaders(assigned_members_df, team_leader_emails, client_secret_file, sender_email):
    scopes = ['https://www.googleapis.com/auth/gmail.send']
    service = get_gmail_service(client_secret_file, scopes)
    
    for _, row in assigned_members_df.iterrows():
        team_assigned = row['Team Assigned']
        team_leader_email = team_leader_emails[team_assigned]
        subject = "New Team Member"
        message_text = f"Dear Team Leader,\n\nI'm pleased to inform you that: {row['First Name']} {row['Last Name']} ({row['Email Address']}) has joined our team, ({row['Team Assigned']}).\n\nPlease update the team roster to include this new addition. Thank you for your attention to this matter.\n\nBest regards,\nAutomated System\nEcoCAR"
        message = create_message(sender_email, team_leader_email, subject, message_text)
        send_email(service, 'me', message)

In [18]:
# Call the function with your dataframe 'df'
notify_team_leaders(assigned_members_df, team_leader_emails, client_secret_file, sender_email)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=171250538774-1hv52jle9rmtc3hpkfhttgs7sdr0ts8d.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A49682%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=sgTil4ze5FjT6p0mBC6mr1PpehFxHr&access_type=offline
Email sent to 18e1b8daecc6a32c


In [19]:
def congratulate_new_member(assigned_members_df, client_secret_file, sender_email):
    scopes = ['https://www.googleapis.com/auth/gmail.send']
    service = get_gmail_service(client_secret_file, scopes)
    
    for _, row in assigned_members_df.iterrows():
        first_name = row['First Name']
        last_name = row['Last Name']
        email_address = row['Email Address']
        team_name = row['Team Assigned']
        
        subject = "Welcome to the Team!"
        message_text = (
            f"Dear {first_name} {last_name},"
            + f"\n\nCongratulations on joining our team, {team_name}! " 
            + f"We are thrilled to have you with us and look forward to achieving great things together."
            + f"\n\nPlease remember to check our team calendar for upcoming meetings and events. " 
            + f"Staying informed will help us all stay on track and make the most out of our collaboration."
            + f"\n\nWelcome aboard!\n\nBest regards,\nKonsing Ham Lopez\nEco Car Team Leader\n{sender_email}"
        )
        message = create_message(sender_email, email_address, subject, message_text)
        send_email(service, 'me', message)

In [20]:
# Send email to student
congratulate_new_member(assigned_members_df, client_secret_file, sender_email)

Email sent to 18e1b8db15c36333


In [21]:
# Code to send email to unassigned members, (tell them, they're on waitlist)
def notify_unassigned_members(unnassigned_members_df, client_secret_file, sender_email):
    scopes = ['https://www.googleapis.com/auth/gmail.send']
    service = get_gmail_service(client_secret_file, scopes)
    
    for _, row in unnassigned_members_df.iterrows():
        first_name = row['First Name']
        last_name = row['Last Name']
        email_address = row['Email Address']
        team_name = row['Team Assigned']
    
        subject = "Update on Team Assignment"
        message_text = (
            f"Dear {first_name} {last_name},\n\n"
            + f"We have received your preferences for team assignments, "
            + f"and we wanted to provide you with an update. "
            + f"Unfortunately, due to a high level of interest and limited availability, "
            + f"we are unable to place you on any of your preferred teams at this time."
            + f"\n\nWe understand this may be disappointing, but you have been placed on our wait list. "
            + f"Should a slot open up on one of your preferred teams, we will reach out to you immediately.\n\n" 
            + f"We appreciate your understanding and patience. If you have any questions or wish to discuss other "
            + f"ways you can contribute in the meantime, please do not hesitate to contact us.\n\n" 
            + f"Best regards,\n" 
            + f"EcoCAR Automated System"
        )
    message = create_message(sender_email, email_address, subject, message_text)
    send_email(service, 'me', message)

In [22]:
# Call the function with your unassigned members dictionary
notify_unassigned_members(unnassigned_members_df, client_secret_file, sender_email)

Email sent to 18e1b8db10b326a3


In [23]:
"""This part of the script will check if the member has been assigned a team
    if not ignore them and move on 
Then, it checks if they have been assigned a team yet (check if the 'Team' column is empty)
    if so, move on
Check if they have been accepted or declined 
    if declined, TODO (undecided)
    If they have been accepted split the team assigned entry using the - character, and update the 'Team' and 'Subteam' column """


#Stores changes for batch update
update_data = []

#Appends the data to the update_data list to push onto the google sheet
def append_data(index_in_df, dataArray, colArr): 
    update_data.append({
            'range': f'{colArr[0]}{index_in_df}:{colArr[1]}{index_in_df}',  # Writing to column J through K 
            'values': [[dataArray[0], dataArray[1]]]
        })
    whole_df.at[index_in_df - 2, 'Team'] = dataArray[0]
    whole_df.at[index_in_df - 2, 'Sub-Team'] = dataArray[1]


def check_subteam_acceptance(service_account_file, spreadsheet_id):
    #Write team assignments to the Google Sheet based on assigned members dataframe."""
    creds = service_credentials.from_service_account_file(service_account_file, scopes=['https://www.googleapis.com/auth/spreadsheets'])
    service = build('sheets', 'v4', credentials=creds)

    for index, row in whole_df.iterrows():
        #At this point, they are not part of a team and a sublead determines whether they get accepted or not
        if row['Accept/Decline'] == "Accept":
            team_assigned = row['Team Assigned'].split("-")
            append_data(index + 2, team_assigned, ['J', 'K'])
        else: 
            #Declined
            pass
            #TODO

    # Batch update the values
    body = {
        'valueInputOption': 'USER_ENTERED',
        'data': update_data
    }
    service.spreadsheets().values().batchUpdate(spreadsheetId=spreadsheet_id, body=body).execute()

In [24]:
check_subteam_acceptance(SERVICE_ACCOUNT_FILE, SPREADSHEET_ID)